In [1]:
!pip install transformers datasets torch accelerate gradio -q

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer, pipeline
)

In [3]:
from datasets import Dataset
import gradio as gr
import torch
import pandas as pd



In [4]:
sample_text = "Large Language Models generate human-like text."

# Tokenizer and model (DistilBERT for simplicity)
tokenizer_demo = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_demo = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
tokens = tokenizer_demo.tokenize(sample_text)
print(" Tokens:", tokens)

 Tokens: ['large', 'language', 'models', 'generate', 'human', '-', 'like', 'text', '.']


In [6]:
input_ids = tokenizer_demo(sample_text, return_tensors="pt").input_ids
embeddings = model_demo.distilbert.embeddings.word_embeddings(input_ids)
print("Embedding shape:", embeddings.shape)

Embedding shape: torch.Size([1, 11, 768])


In [7]:
data = pd.read_csv('/content/train.csv')

In [10]:
# Ensure all object columns are treated as strings to avoid conversion errors
for col in data.select_dtypes(include='object').columns:
    data[col] = data[col].astype(str)

dataset = Dataset.from_list(data.to_dict('records'))

# Tokenization / preprocessing
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, use_auth_token=False)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
def preprocess(example):
    inp = f"Instruction: {example['instruction']} Input: {example['input']}"
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset.map(preprocess, batched=False)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_name, use_auth_token=False)

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4715: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
training_args = TrainingArguments(
    output_dir="./t5_instruction_finetune",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no",
    push_to_hub=False,        # <— prevents token prompt
    report_to=[]              # <— disables any reporting
)

In [25]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_data)
trainer.train()
trainer.save_model("./t5_instruction_model")

Step,Training Loss
500,1.811500
1000,1.802300
1500,1.815100
2000,1.802800
2500,1.970200
3000,1.991300
3500,2.053800
4000,2.106600
4500,2.085200
5000,2.125700


In [26]:
pipe = pipeline("text2text-generation", model="./t5_instruction_model", tokenizer=model_name)

Device set to use cuda:0


In [27]:
def generate_text(instruction, user_input):
    prompt = f"Instruction: {instruction} Input: {user_input}"
    result = pipe(prompt, max_length=80)[0]['generated_text']
    return result

demo = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Instruction (e.g. Summarize the text)"),
            gr.Textbox(label="Input Text")],
    outputs=gr.Textbox(label="Generated Output"),
    title=" Mini GenAI Instruction Model (Token-Free)",
    description="Fine-tuned T5-small model trained"
)

In [28]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de0d765ae04e3e3134.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [29]:
import shutil
from google.colab import files

# Define the directory to zip
dir_to_zip = "/content/t5_instruction_model"
output_filename = "t5_instruction_model"

# Create a zip archive
shutil.make_archive(output_filename, 'zip', dir_to_zip)

# Provide the zip file for download
files.download(f"{output_filename}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>